In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=238c6dfacc836a416a2efa07d0abe30b5b7b247d664a4c5136809d13eda503f8
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
# Spar context létrehozása.
# Mindenképp csak egyszer futtassuk le!!!
# Ezen keresztül tudjuk elérni az RDD-kehz tartozó low level API-t.

from pyspark import SparkConf
from pyspark import SparkContext

conf = SparkConf()
sc = SparkContext(conf=conf)

In [3]:
# Lemezen lévő fájlból is készíthetünk RDD-t
# Az alábbi kód kírja a szöveg első két sorát:

lines = sc.textFile('/content/drive/MyDrive/Colab Notebooks/1-2-3/alkotmany.txt').take(2)

print(lines)

['Mi, a magyar nemzet tagjai, az új évezred kezdetén, felelősséggel minden magyarért, kinyilvánítjuk az alábbiakat:', 'Büszkék vagyunk arra, hogy Szent István királyunk ezer évvel ezelőtt szilárd alapokra helyezte a magyar államot, és hazánkat a keresztény Európa részévé tette.']


In [5]:
# Feladat: Grep külön függvénnyel

def filter_row(line):
    grepWord = 'magyar'
    if grepWord in line:
        return True
    else:
        return False


lines = sc.textFile('/content/drive/MyDrive/Colab Notebooks/1-2-3/alkotmany.txt')\
.filter(filter_row)

lines.take(2)

['Mi, a magyar nemzet tagjai, az új évezred kezdetén, felelősséggel minden magyarért, kinyilvánítjuk az alábbiakat:',
 'Büszkék vagyunk arra, hogy Szent István királyunk ezer évvel ezelőtt szilárd alapokra helyezte a magyar államot, és hazánkat a keresztény Európa részévé tette.']

In [ ]:
# Feladat: Grep

grepWord = 'magyar'

lines = sc.textFile('alkotmany.txt')\
.filter(lambda line: grepWord in line)

lines.take(2)

['Mi, a magyar nemzet tagjai, az új évezred kezdetén, felelősséggel minden magyarért, kinyilvánítjuk az alábbiakat:',
 'Büszkék vagyunk arra, hogy Szent István királyunk ezer évvel ezelőtt szilárd alapokra helyezte a magyar államot, és hazánkat a keresztény Európa részévé tette.']

In [ ]:
lines = sc.textFile('alkotmany.txt')\
.map(lambda l: l.split(" "))

lines.take(2)

[['Mi,',
  'a',
  'magyar',
  'nemzet',
  'tagjai,',
  'az',
  'új',
  'évezred',
  'kezdetén,',
  'felelősséggel',
  'minden',
  'magyarért,',
  'kinyilvánítjuk',
  'az',
  'alábbiakat:'],
 ['Büszkék',
  'vagyunk',
  'arra,',
  'hogy',
  'Szent',
  'István',
  'királyunk',
  'ezer',
  'évvel',
  'ezelőtt',
  'szilárd',
  'alapokra',
  'helyezte',
  'a',
  'magyar',
  'államot,',
  'és',
  'hazánkat',
  'a',
  'keresztény',
  'Európa',
  'részévé',
  'tette.']]

In [ ]:
lines = sc.textFile('alkotmany.txt')\
.flatMap(lambda l: l.split(" "))\
.sortBy(lambda word: word)

lines.take(10)

['1944.',
 '1949.',
 '1956-os',
 '1990.',
 'Alaptörvényünk',
 'Becsüljük',
 'Bízunk',
 'Büszkék',
 'Büszkék',
 'Büszkék']

In [ ]:
# Kulcs-érték párok készítése
# Műveletek kulcs-érték párokon: reduceByKey, groupByKey, countByKey

lines = sc.textFile('alkotmany.txt')\
.flatMap(lambda l: l.split(" "))\
.map(lambda w: (w.lower()[0], w.lower()))

lines.take(10)

[('m', 'mi,'),
 ('a', 'a'),
 ('m', 'magyar'),
 ('n', 'nemzet'),
 ('t', 'tagjai,'),
 ('a', 'az'),
 ('ú', 'új'),
 ('é', 'évezred'),
 ('k', 'kezdetén,'),
 ('f', 'felelősséggel')]

In [ ]:
# Feladat: WordCount
# Kulcs alapján aggregáló függvények: reduceByKey, aggregateByKey, groupByKey, sortByKey

lines = sc.textFile('alkotmany.txt')\
.flatMap(lambda l: l.split(" "))\
.map(lambda l: l.replace(':',"").replace(",","").replace(".", "").replace("!",""))\
.map(lambda w: (w.lower(), 1))\
.reduceByKey(lambda a, b: a + b)\
.sortBy(lambda x: x[1], False)

lines.take(5)

[('a', 41), ('és', 24), ('az', 19), ('hogy', 17), ('valljuk', 9)]

In [ ]:
#2 Word count #2 megoldas
def sajat_sum(vals):
    osszeg = 0
    for i in vals:
        osszeg += i
    return osszeg

lines = sc.textFile('alkotmany.txt')\
.flatMap(lambda l: l.split(" "))\
.map(lambda l: l.replace(':',"").replace(",","").replace(".", "").replace("!",""))\
.map(lambda w: (w.lower(), 1))\
.groupByKey()\
.mapValues(sajat_sum)\
.sortBy(lambda t: t[1], False)

lines.take(5)

[('a', 41), ('és', 24), ('az', 19), ('hogy', 17), ('valljuk', 9)]

In [ ]:
# feladat Adjuk meg a legtobb karakterbol allo szo hosszat

lines = sc.textFile('alkotmany.txt')\
.flatMap(lambda x: x.split(' '))\
.map(lambda w: len(w))

lines.max()


18

In [ ]:
# Feladat: Melyik a leghosszabb szó és milyen hosszú?

lines = sc.textFile("alkotmany.txt")\
.flatMap(lambda l : l.split(" "))\
.map(lambda w : (w,len(w)))\
.reduce(lambda a, b: a if a[1] > b[1] else b)

print(lines)

('elhivatottságában.', 18)


In [ ]:
# Feladat: MaxLength1
# Adjuk meg kezdőbetűnként a leghosszabb szó hosszát

lines = sc.textFile("alkotmany.txt")\
.flatMap(lambda l : l.split(" "))\
.map(lambda w: (w[0].lower(), len(w)))\
.reduceByKey(lambda a,b: a if a > b else b)\
.sortBy(lambda t: t[0])

lines.take(5)

[('1', 7), ('a', 15), ('b', 13), ('c', 6), ('d', 12)]

In [9]:
#feladat> Wordmean> Adjuk meg a szavak atlagos hosszat

lines = sc.textFile('/content/drive/MyDrive/Colab Notebooks/1-2-3/alkotmany.txt')\
.flatMap(lambda x: x.split(' '))\
.map(lambda w: len(w) )

#print(lines.mean())
print(lines.sum()/lines.count())

6.974947807933194
6.974947807933194


In [ ]:
# 5.feladat

import random

NUM_SAMPLE = 100000000

def inside(p):
  x,y = random.random(), random.random()
  return x*x+y*y<=1

#szamok = sc.parallelize(range(NUM_SAMPLE))\
#.filter(inside).count()

szamok = sc.parallelize(range(NUM_SAMPLE))\
.map(inside)\
.filter(lambda x:x)\
.count()

#szamok.take(5)
print(4*(szamok/NUM_SAMPLE))

3.14179492


In [ ]:
inside(10)

True

In [10]:
# Feladat MaxTemp> Napi maximalis homerseklet szamolo

data = sc.textFile("/content/drive/MyDrive/Colab Notebooks/1-2-3/tempBudapestMeteoBlue.csv")\
.map(lambda t: t.split(","))\
.filter(lambda l: l[0].startswith('2020'))\
.map(lambda t: (t[0].split("T"), t[1]))\
.map(lambda t: (t[0][0], (t[0][1], t[1])))\
.reduceByKey(lambda a,b: a if float(a[1]) > float(b[1]) else b)

data.collect()

[('20200916', ('1600', '31.431458')),
 ('20200917', ('1500', '30.111458')),
 ('20200918', ('1600', '22.861458')),
 ('20200919', ('1600', '23.311459')),
 ('20200920', ('1700', '25.321459')),
 ('20200921', ('1300', '27.171457')),
 ('20200922', ('1600', '27.831459')),
 ('20200923', ('1400', '27.131458')),
 ('20200924', ('1500', '26.421457')),
 ('20200926', ('0000', '17.621458')),
 ('20200927', ('1500', '18.401459')),
 ('20200928', ('1500', '19.731459')),
 ('20200914', ('1700', '30.651459')),
 ('20200915', ('1600', '32.06146')),
 ('20200925', ('1800', '22.981459'))]

In [ ]:
# Accumulator

elofordulas = sc.accumulator(0)
#elofordulas = 0

def countWord(w):
  if 'MAGYAR' in w.upper():
    elofordulas.add(1)

lines = sc.textFile('alkotmany.txt')\
.flatMap(lambda l: l.split(' '))\
.foreach(countWord)

print(elofordulas)

12


In [15]:
# Broadcast

keresendo = ['MAGYAR']
br_keresendo = sc.broadcast(keresendo) # arra jó, hogy csak egyszer küldi el az egységeknek és nem többször
elofordulas = sc.accumulator(0)

def countWord(w):
  if br_keresendo.value[0] in w.upper():
    elofordulas.add(1)

lines = sc.textFile('/content/drive/MyDrive/Colab Notebooks/1-2-3/alkotmany.txt')\
.flatMap(lambda l: l.split(' '))\
.foreach(countWord)

print(elofordulas)

12


In [ ]:
# Feladat: Maxtemp 2
# Oldjuk meg a MaxTemp feladatot úgy, hogy közben összeszámoljuk hány nap volt magasabb a hőmérséklet 30 foknál

myacc = sc.accumulator(0)

def myadd(t):
    if float(t[1][1]) > 30:
        myacc.add(1)
    return t

data = sc.textFile("tempBudapestMeteoBlue.csv")\
.map(lambda t: t.split(","))\
.filter(lambda l: l[0].startswith('2020'))\
.map(lambda t: (t[0].split("T"), t[1]))\
.map(lambda t: (t[0][0], (t[0][1], t[1])))\
.reduceByKey(lambda a,b: a if float(a[1]) > float(b[1]) else b)\
.map(myadd)

print(data.collect())
print(myacc.value)

[('20200916', ('1600', '31.431458')), ('20200917', ('1500', '30.111458')), ('20200918', ('1600', '22.861458')), ('20200919', ('1600', '23.311459')), ('20200920', ('1700', '25.321459')), ('20200921', ('1300', '27.171457')), ('20200922', ('1600', '27.831459')), ('20200923', ('1400', '27.131458')), ('20200924', ('1500', '26.421457')), ('20200926', ('0000', '17.621458')), ('20200927', ('1500', '18.401459')), ('20200928', ('1500', '19.731459')), ('20200914', ('1700', '30.651459')), ('20200915', ('1600', '32.06146')), ('20200925', ('1800', '22.981459'))]
4
